In [8]:
from minitorch.scalar import Scalar, Neg, Mul

In [4]:
a = Scalar(3.0)
b = Scalar(4.0)

y = a + b

In [5]:
y.history

ScalarHistory(last_fn=<class 'minitorch.scalar_functions.Add'>, ctx=Context(no_grad=False, saved_values=()), inputs=[Scalar(3.000000), Scalar(4.000000)])

In [11]:
z = Neg.apply(a)
z.history

ScalarHistory(last_fn=<class 'minitorch.scalar_functions.Neg'>, ctx=Context(no_grad=False, saved_values=()), inputs=[Scalar(3.000000)])

In [14]:
out = Mul.apply(b, z)

print(out.history)

ScalarHistory(last_fn=<class 'minitorch.scalar_functions.Mul'>, ctx=Context(no_grad=False, saved_values=(4.0, -3.0)), inputs=[Scalar(4.000000), Scalar(-3.000000)])


In [19]:
from project.interface.plots import plot_function
from minitorch.scalar import ScalarFunction
import math
import minitorch


class Sin(ScalarFunction):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return math.sin(x)

    @staticmethod
    def backward(ctx, d):
        (x,) = ctx.saved_values
        f_prime = math.cos(x)
        return f_prime * d


def d_call(x):
    ctx = minitorch.Context()
    Sin.forward(ctx, x)
    return Sin.backward(ctx, 1)

plot_function("f(x) = sin(x)", lambda x: Sin.apply(x).data)
plot_function("1 * f'(x) = cos(x)", d_call)

In [25]:
class Mul(ScalarFunction):
    @staticmethod
    def forward(ctx, x, y):
        ctx.save_for_backward(x, y)
        return x * y

    @staticmethod
    def backward(ctx, d):
        # Compute f'_x(x, y) * d, f'_y(x, y) * d
        x, y = ctx.saved_values
        f_x_prime = y
        f_y_prime = x
        return f_x_prime * d, f_y_prime * d

plot_function("f(x) = x * 3.0", lambda x: Mul.apply(x, 3.0).data)